# repro-rpg のデザイン案 ver. 0.2
以下Battle Classに隠蔽されているコード
遊ぶときは
```
imoprt repro-rpg
```
とかになる


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import random
from abc import ABCMeta, abstractmethod

In [2]:
class Skill():
    
    @abstractmethod
    def __init__(self):
        self.name = ''
        self.type = 'normal'
        self.value = 1
    
    def __process__(self):
        pass

In [3]:
class NormalAttack(Skill):
    def __init__(self):
        self.name = '攻撃'
        self.type = 'normal'
        self.value = 30
        
    def description(self):
        print('攻撃')
        print('MP 消費なし')
        print('1体に攻撃する')
        print('攻撃力が上がる')

    def process(self):
        pass

In [4]:
class StrongAttack(Skill):
    def __init__(self):
        self.name = '強攻撃'
        self.type = 'normal'
        self.value = 100
        
    def description(self):
        print('強攻撃')
        print('MP 消費なし')
        print('1体に攻撃する')
        print('すばやさがさがる')
        
        
    def process(self):
        pass

In [5]:
class Heal(Skill):
    def __init__(self):
        self.name = '回復'
        self.type = 'recover'
        self.value = 20
        
    def description(self):
        print('回復')
        print('MP 消費なし')
        print('1体を回復する')
        print('攻撃力がさがる')
        
        
    def process(self):
        pass

In [6]:
class BattleUnit():
    def __init__(self):
        self.health = 10
        self.attack = 10
        self.defence = 2
        self.speed = 5
        self.isPlayer = True
        self.skills = None

In [7]:
class Player(BattleUnit):
    
    def __init__(self, level=10):
        self.name = 'オレサマ'
        self.health = 10 + level * 0.1
        self.attack = 8 + level * 0.1
        self.defence = 1 + level * 0.1
        self.speed = 5 + level * 0.1
        self.isPlayer = True
        self.skills =[NormalAttack(), StrongAttack(), Heal()]
        

In [8]:
class Enemy(BattleUnit):
    
    def __init__(self, level=1):
        self.name = '敵 lv.' + str(level)
        self.health = 10 + level * 0.2
        self.attack = 5 + level * 0.2
        self.defence = 1 + level * 0.2
        self.speed = 2 + level * 0.2
        self.isPlayer = False
        self.skills =[NormalAttack()]
    
    def algorithm(self, current_battle_info):
        skill_list = [skill.name for skill in current_battle_info[0].skills]
    
        enemies = [unit for unit in current_battle_info if  unit.isPlayer]
        enemy_index = np.argmin([enemy.health for enemy in enemies])
        target_index = current_battle_info.index(enemies[enemy_index])
    
        return 0, [target_index]


In [9]:
class DangerousEnemy(BattleUnit):
    
    def __init__(self, level=1):
        self.name = '危ない敵 lv.' + str(level)
        self.health = 1 + level * 0.02
        self.attack = 10 + level * 0.2
        self.defence = 1 + level * 0.2
        self.speed = 1 + level * 0.02
        self.isPlayer = False
        self.skills =[NormalAttack()]
    
    def algorithm(self, current_battle_info):
            
        skill_list = [skill.name for skill in current_battle_info[0].skills]
    
        enemies = [unit for unit in current_battle_info if unit.isPlayer]
        enemy_index = np.argmin([enemy.health for enemy in enemies])
        target_index = current_battle_info.index(enemies[enemy_index])
    
        return 0, [target_index]


In [10]:
class Battle():
    def __init__(self):
        self.battle_units = []
        self.finished = False
        self.player_win = False
        
    def basic_algorithm(self):
        skill = NormalAttack()
        return skill
        
    def appry_unit(self, unit):
        self.battle_units.append(unit)
    
    def current_battle_info(self):
        return self.battle_units # tentative
    
    def skill_list(self):
        return [skill.name for skill in self.battle_units[0].skills]

    def apply_algorithm(self, func):
        self.algorithm = func
    
    
    def process_turn(self):
        
        random.shuffle(self.battle_units)
        self.battle_units = sorted(self.battle_units, key=lambda t: (t.speed), reverse=True)
        #print(self.battle_units)
        
        for unit in self.battle_units:
            print(unit.name + '  HP :' + str(round(unit.health)))
        
        for self.active_unit in self.battle_units:
            
            
            used_skill = None
            targets = None
            if self.active_unit.isPlayer:
                index_skill, targets = self.algorithm(self.current_battle_info())
            else:
                index_skill, targets = self.active_unit.algorithm(self.current_battle_info())

            used_skill = self.active_unit.skills[index_skill]
            print(self.active_unit.name + 'の' + used_skill.name)
    
            # Process for skills
            if used_skill.name == '攻撃':
                if len(targets) != 1:
                    print('ターゲット指定が誤っています.')
                    targets = []
                for target in targets:
                    damage = self.active_unit.attack * used_skill.value * 0.01 - self.battle_units[target].defence
                    damage = np.maximum(np.random.randint(0,2), damage)             
                    self.battle_units[target].health -= damage
                    print(self.battle_units[target].name+'に'+str(round(damage))+'のダメージ')
                    self.active_unit.attack *= 1.2
                    print(self.active_unit.name + 'の攻撃力があがった')
            
            elif used_skill.name == '回復':
                if len(targets) != 1:
                    print('ターゲット指定が誤っています.') 
                    targets = []
                for target in targets:
                    print('aa' + used_skill.name)
                    value = used_skill.value 
                    value = np.maximum(np.random.randint(0,2), value)             
                    self.battle_units[target].health += value
                    print(self.battle_units[target].name+'は'+str(round(value))+'回復した')
                    self.active_unit.attack *= 0.7
                    print(self.active_unit.name + 'の攻撃力が下がった')
            
            elif used_skill.name == '強攻撃':
                if len(targets) != 1:
                    print('ターゲット指定が誤っています.')    
                    targets = []
                for target in targets:
                    damage = self.active_unit.attack * used_skill.value * 0.01 - self.battle_units[target].defence
                    damage = np.maximum(np.random.randint(0,2), damage)             
                    self.battle_units[target].health -= damage
                    print(self.battle_units[target].name+'に'+str(round(damage))+'のダメージ')
                    self.active_unit.speed *= 0.7
                    print(self.active_unit.name + 'のすばやさが下がった')
      
            else:
                pass
            
            for i in reversed(range(len(self.battle_units))):
                if self.battle_units[i].health <= 0:
                    print(self.battle_units[i].name + 'は死んだ')
                    del self.battle_units[i]
            
            alives = [unit.isPlayer for unit in self.battle_units]
            n_players = np.count_nonzero(alives)
            n_enemies = len(alives) - n_players
            
            if n_players == 0:
                print('Game Over')
                self.finished = True
                break
            if n_enemies == 0:
                print('Player Win!')
                self.finished = True
                self.player_win = True
                break
                
                
    def process(self):
        
        i_turn = 1
        while not self.finished:
            print( 'ターン' + str(i_turn))
            self.process_turn()
            i_turn += 1
            if i_turn > 50:
                break

## ユーザーが工夫して書くアルゴリズム

 - current_battle_info を受け取って, 自分の使いたいSkillのindexを返す
 - 現状はcurrent_battle_infoはすべての情報を持っているが, これはBattleクラス内で情報を欠落させて渡す

In [11]:
# User should write this function, then apply battle class

def my_algorithm(current_battle_info):
    
    skill_list = [skill.name for skill in current_battle_info[0].skills]
    
    enemies = [unit for unit in current_battle_info if not unit.isPlayer]
    target_index = current_battle_info.index(enemies[0])
    return 0, [target_index] 

In [12]:
# 1戦闘のスタート

player = Player(level=10)
enemy1 = Enemy(level=10)
enemy2 = DangerousEnemy(level=10)


b = Battle()
b.appry_unit(player)
b.appry_unit(enemy1)
b.appry_unit(enemy2)

In [13]:
b.apply_algorithm(my_algorithm)

In [14]:
b.process()

ターン1
オレサマ  HP :11
敵 lv.10  HP :12
危ない敵 lv.10  HP :1
オレサマの攻撃
敵 lv.10に0.0のダメージ
オレサマの攻撃力があがった
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.10の攻撃力があがった
危ない敵 lv.10の攻撃
オレサマに2.0のダメージ
危ない敵 lv.10の攻撃力があがった
ターン2
オレサマ  HP :8.0
敵 lv.10  HP :12.0
危ない敵 lv.10  HP :1
オレサマの攻撃
敵 lv.10に0.0のダメージ
オレサマの攻撃力があがった
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.10の攻撃力があがった
危ない敵 lv.10の攻撃
オレサマに2.0のダメージ
危ない敵 lv.10の攻撃力があがった
ターン3
オレサマ  HP :5.0
敵 lv.10  HP :12.0
危ない敵 lv.10  HP :1
オレサマの攻撃
敵 lv.10に1.0のダメージ
オレサマの攻撃力があがった
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.10の攻撃力があがった
危ない敵 lv.10の攻撃
オレサマに3.0のダメージ
危ない敵 lv.10の攻撃力があがった
ターン4
オレサマ  HP :1.0
敵 lv.10  HP :11.0
危ない敵 lv.10  HP :1
オレサマの攻撃
敵 lv.10に2.0のダメージ
オレサマの攻撃力があがった
敵 lv.10の攻撃
オレサマに2.0のダメージ
敵 lv.10の攻撃力があがった
オレサマは死んだ
Game Over


## あそびかた
 - まずはInteractiveにコードを書いてみる

In [15]:
# 1戦闘のスタート (コピペ)

player = Player()
enemy1 = Enemy(level=10)
enemy2 = DangerousEnemy(level=10)

b = Battle()
b.appry_unit(player)
b.appry_unit(enemy1)
b.appry_unit(enemy2)

In [16]:
b.current_battle_info() # 敵味方の情報が返ってくる

In [17]:
#さっき敗北したアルゴリズムを再度Apply
# User should write this function, then apply battle class

def my_algorithm(current_battle_info):
    
    skill_list = [skill.name for skill in current_battle_info[0].skills]
    
    enemies = [unit for unit in current_battle_info if not unit.isPlayer]
    target_index = current_battle_info.index(enemies[0])
    return 0, [target_index] 

b.apply_algorithm(my_algorithm)

In [18]:
# 1ターンずつ動かす.
b.process_turn()

オレサマ  HP :11
敵 lv.10  HP :12
危ない敵 lv.10  HP :1
オレサマの攻撃
敵 lv.10に1.0のダメージ
オレサマの攻撃力があがった
敵 lv.10の攻撃
オレサマに0.0のダメージ
敵 lv.10の攻撃力があがった
危ない敵 lv.10の攻撃
オレサマに2.0のダメージ
危ない敵 lv.10の攻撃力があがった


In [19]:
b.current_battle_info()

In [20]:
print(b.current_battle_info()[2].name)
print(b.current_battle_info()[2].attack)
print(b.current_battle_info()[2].defence)
print(b.current_battle_info()[2].health)

危ない敵 lv.10
14.399999999999999
3.0
1.2


めっちゃたかい攻撃力しているので, こいつから殺そうという発想になる

In [21]:
# 1戦闘のスタート

player = Player()
enemy1 = Enemy(level=10)
enemy2 = DangerousEnemy(level=10)

b = Battle()
b.appry_unit(player)
b.appry_unit(enemy1)
b.appry_unit(enemy2)

In [22]:
# User should write this function, then apply battle class

def my_algorithm(current_battle_info):
    
    skill_list = [skill.name for skill in current_battle_info[0].skills]
    
    enemies = [unit for unit in current_battle_info if not unit.isPlayer]
    enemy_index = np.argmax([enemy.attack for enemy in enemies])  #敵で攻撃力が高いやつを対象にする
    target_index = current_battle_info.index(enemies[enemy_index])
    
    return 1, [target_index]

In [23]:
b.apply_algorithm(my_algorithm)

In [24]:
b.process()

ターン1
オレサマ  HP :11
敵 lv.10  HP :12
危ない敵 lv.10  HP :1
オレサマの強攻撃
危ない敵 lv.10に6.0のダメージ
オレサマのすばやさが下がった
危ない敵 lv.10は死んだ
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.10の攻撃力があがった
ターン2
オレサマ  HP :10.0
敵 lv.10  HP :12
オレサマの強攻撃
敵 lv.10に6.0のダメージ
オレサマのすばやさが下がった
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.10の攻撃力があがった
ターン3
敵 lv.10  HP :6.0
オレサマ  HP :9.0
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.10の攻撃力があがった
オレサマの強攻撃
敵 lv.10に6.0のダメージ
オレサマのすばやさが下がった
敵 lv.10は死んだ
Player Win!


## 勝てた.

### その他の要素
 - 回復も入れてみた
 - ただし, ステータス下がるようにしたら回復のメリット死んでいる気がする
  - むしろ早く倒せなくなった.
 - MPとかいれなければいけないのか
 - unitをリストで渡しているだけなので, 何ターン目かみたいな情報は全然ないな.

In [25]:
# 1戦闘のスタート

player = Player()
enemy1 = Enemy(level=20)
enemy2 = DangerousEnemy(level=20)

b = Battle()
b.appry_unit(player)
b.appry_unit(enemy1)
b.appry_unit(enemy2)

In [26]:
# User should write this function, then apply battle class

def my_algorithm(current_battle_info):
    
    player = [unit for unit in current_battle_info if unit.isPlayer][0]
    player_index = current_battle_info.index(player)
    player_skill_names = [skill.name for skill in player.skills]
    
    if player.health < 3:
        return player_skill_names.index('回復'), [player_index]
        
    enemies = [unit for unit in current_battle_info if not unit.isPlayer]
    enemy_index = np.argmin([enemy.health for enemy in enemies])
    target_index = current_battle_info.index(enemies[enemy_index])
    
    return 1, [target_index]

In [27]:
b.apply_algorithm(my_algorithm)

In [28]:
b.process()

ターン1
敵 lv.20  HP :14
オレサマ  HP :11
危ない敵 lv.20  HP :1
敵 lv.20の攻撃
オレサマに1.0のダメージ
敵 lv.20の攻撃力があがった
オレサマの強攻撃
危ない敵 lv.20に4.0のダメージ
オレサマのすばやさが下がった
危ない敵 lv.20は死んだ
ターン2
敵 lv.20  HP :14
オレサマ  HP :10.0
敵 lv.20の攻撃
オレサマに1.0のダメージ
敵 lv.20の攻撃力があがった
オレサマの強攻撃
敵 lv.20に4.0のダメージ
オレサマのすばやさが下がった
ターン3
敵 lv.20  HP :10.0
オレサマ  HP :9.0
敵 lv.20の攻撃
オレサマに2.0のダメージ
敵 lv.20の攻撃力があがった
オレサマの強攻撃
敵 lv.20に4.0のダメージ
オレサマのすばやさが下がった
ターン4
敵 lv.20  HP :6.0
オレサマ  HP :7.0
敵 lv.20の攻撃
オレサマに3.0のダメージ
敵 lv.20の攻撃力があがった
オレサマの強攻撃
敵 lv.20に4.0のダメージ
オレサマのすばやさが下がった
ターン5
敵 lv.20  HP :2.0
オレサマ  HP :4.0
敵 lv.20の攻撃
オレサマに4.0のダメージ
敵 lv.20の攻撃力があがった
オレサマの回復
aa回復
オレサマは20回復した
オレサマの攻撃力が下がった
ターン6
敵 lv.20  HP :2.0
オレサマ  HP :21.0
敵 lv.20の攻撃
オレサマに5.0のダメージ
敵 lv.20の攻撃力があがった
オレサマの強攻撃
敵 lv.20に1.0のダメージ
オレサマのすばやさが下がった
ターン7
敵 lv.20  HP :1.0
オレサマ  HP :16.0
敵 lv.20の攻撃
オレサマに6.0のダメージ
敵 lv.20の攻撃力があがった
オレサマの強攻撃
敵 lv.20に1.0のダメージ
オレサマのすばやさが下がった
敵 lv.20は死んだ
Player Win!
